In [1]:
# blab init
import blab
startup_notebook = blab.blab_startup()
%run $startup_notebook

blab init
blab found local libs at /media/me/Data_slow/Dropbox/31_Projekte/01_Python/libs
Start Time: 20:52:02


time: 453 ms


# Search Strings Fast
* `fast_startswith`: Searches string columns for matching beginnings.<br>
   Like pandas str.startswith(), but much faster for large amounts of data, and it returns the matching fragment.
* `fast_endswith`: Searches string columns for matching endings.

In [2]:
#import numpy      as np
import pandas     as pd 
import bpyth      as bpy
import pandasklar as pak 
pak.Config.set('VERBOSE', True)

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

time: 955 ms


## fast_startswith()

In [3]:
?pak.fast_startswith

time: 43.8 ms


Signature:
pak.fast_startswith(
    df,
    col_search,
    col_found,
    searchfor,
    find_longest=True,
    find_identical=True,
)
Docstring:
Searches string columns for matching beginnings.
Like pandas str.startswith(), but much faster for large amounts of data,
and it returns the matching fragment. 
* col_search:     Name of the column to be searched
* col_found:      Names of the column into which the result is to be written
* searchfor:      Series or List of strings to be searched for
* find_longest:   Should the longest substring be given as the result? Otherwise the shortest.
* find_identical: Should it be counted as a result if a string matches completely?
File:      /media/me/Data_slow/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/string.py
Type:      function


In [4]:
# create data to search in
size = 300000 # REDUCE THIS IF NECESSARY
manystrings = pd.DataFrame(pak.random_series(size,'string',len_min=10, len_max=10, name='U'))
print(manystrings.shape[0], 'rows to search in')
manystrings.head(3)

300000 rows to search in


,U
0,MKböSYXrrJ
1,sAöAThCTOd
2,ÖuREsW9YxI


time: 6.47 s


In [5]:
# create data to search
search_me   = pak.random_series( int(size/10),'string',len_min=2, len_max=4)
print(search_me.shape[0], 'strings to search for')
search_me.head(3)

30000 strings to search for


0    Fl
1    bH
2    og
Name: random, dtype: string

time: 342 ms


In [6]:
%%time 

# Try built-in startswith
# Wall time: 25s @ size = 300000

mask = manystrings.U.str.startswith(tuple(search_me))
result0 = manystrings[mask] # result of the built-in startswith


print('found', result0.shape[0], 'matching rows')
result0.head(3)

found 280601 matching rows
CPU times: user 22.3 s, sys: 61 µs, total: 22.3 s
Wall time: 22.3 s


,U
0,MKböSYXrrJ
1,sAöAThCTOd
2,ÖuREsW9YxI


time: 22.4 s


In [7]:
%%time

# Try fast_startswith
# Wall time: 779 ms @ size = 300000

df = pak.fast_startswith( manystrings, 'U', 'found', search_me ) 
mask = df.found.notnull()
result1 = df[mask]   # result of fast_startswith

print('found', result1.shape[0], 'matching rows')
assert result0.shape[0] == result1.shape[0]
result1.head(3)

found 280601 matching rows
CPU times: user 745 ms, sys: 8.03 ms, total: 753 ms
Wall time: 752 ms


,U,found
0,MKböSYXrrJ,MK
1,sAöAThCTOd,sA
2,ÖuREsW9YxI,Öu


time: 773 ms


### find_identical, find_longest

In [8]:
# create data to search in
size = 100000 
manystrings = pd.DataFrame(pak.random_series(size,'string',len_min=2, len_max=4, name='U'))
print(manystrings.shape[0], 'rows to search in')
manystrings.head(3)

100000 rows to search in


,U
0,B3RD
1,gHuo
2,qäx


time: 1.13 s


In [9]:
# search self for longest substring
df = pak.fast_startswith(manystrings, 'U', 'found_longest', manystrings.U, find_identical=False, find_longest=True) 
mask = df.found_longest.notnull()
self1 = df[mask]   
self1

,U,found_longest
0,B3RD,B3R
1,gHuo,gH
2,qäx,qä
3,iIpW,iI
4,z8U,z8
...,...,...
99995,9zpÜ,9zp
99996,oOQl,oO
99997,kdfE,kd
99998,öYüR,öY


time: 358 ms


In [10]:
# search self for shortest substring
df = pak.fast_startswith(manystrings, 'U', 'found_shortest', manystrings.U, find_identical=False, find_longest=False) 
mask = df.found_shortest.notnull()
self2 = df[mask]   
self2

,U,found_shortest
0,B3RD,B3
1,gHuo,gH
2,qäx,qä
3,iIpW,iI
4,z8U,z8
...,...,...
99995,9zpÜ,9z
99996,oOQl,oO
99997,kdfE,kd
99998,öYüR,öY


time: 352 ms


In [11]:
# compare longest and shortest
result = pd.merge(self1, self2, how='outer')
assert self1.shape[0] == result.shape[0]
assert self2.shape[0] == result.shape[0]
mask = result.found_shortest != result.found_longest

result[mask]

,U,found_longest,found_shortest
0,B3RD,B3R,B3
29,YWvA,YWv,YW
60,O6Mj,O6M,O6
86,aziT,azi,az
94,db8b,db8,db
...,...,...,...
95347,a4eq,a4e,a4
95348,ÖiÖb,ÖiÖ,Öi
95353,öuu0,öuu,öu
95369,Üab7,Üab,Üa


time: 80.8 ms


## fast_endswith()

In [12]:
# create data to search in
size = 100000 # REDUCE THIS IF NECESSARY
manystrings = pd.DataFrame(pak.random_series(size,'string',len_min=10, len_max=10, name='U'))
print(manystrings.shape[0], 'rows to search in')
manystrings.head(3)

100000 rows to search in


,U
0,uNX2jEwtSb
1,Ew99qy8on5
2,cXu934rrmc


time: 2.2 s


In [13]:
# create data to search
search_me   = pak.random_series( int(size/10),'string',len_min=2, len_max=4)
print(search_me.shape[0], 'strings to search for')
search_me.head(3)

10000 strings to search for


0     WwW
1      YQ
2    ÄSi3
Name: random, dtype: string

time: 130 ms


In [14]:
%%time 

# Try built-in endswith
# Wall time: 5.43 s @ size = 100000

mask = manystrings.U.str.endswith(tuple(search_me))
result0 = manystrings[mask] # result of the built-in startswith


print('found', result0.shape[0], 'matching rows')
result0.head(3)

found 57134 matching rows
CPU times: user 5.59 s, sys: 0 ns, total: 5.59 s
Wall time: 5.59 s


,U
0,uNX2jEwtSb
2,cXu934rrmc
3,WlQau4sxX2


time: 5.61 s


In [15]:
%%time

# Try fast_endswith
# Wall time: 307 ms @ size = 100000

df = pak.fast_endswith( manystrings, 'U', 'found', search_me ) 
mask = df.found.notnull()
result1 = df[mask]   # result of fast_startswith

print('found', result1.shape[0], 'matching rows')
assert result0.shape[0] == result1.shape[0]
result1.head(3)

found 57134 matching rows
CPU times: user 279 ms, sys: 8.06 ms, total: 287 ms
Wall time: 286 ms


,U,found
0,uNX2jEwtSb,Sb
2,cXu934rrmc,mc
3,WlQau4sxX2,X2


time: 307 ms
